In [2]:
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
with open("Transaction History.html") as fp:
    soup = BeautifulSoup(fp)

In [4]:
rows = soup.find('div', id='divHist').find('tbody').find_all('tr')

In [6]:
post_date = []
loc = []
typ = []
amt = []

for i in range(1,len(rows)):
    process_row(rows[i])

In [ ]:
def process_rows(row):
    cell = rows[i].find_all('td')
    post_date.append(cell[0].string.strip())
    loc.append(cell[1].string.strip())
    typ.append(cell[3].string.strip())
    amt.append(cell[4].string.strip())

In [7]:
data = {'Date':post_date, 'Location':loc, 'Type':typ, 'Amount':amt}
df = pd.DataFrame(data)
df.head(10)

,Date,Location,Type,Amount
0,08/21/2019 16:40:21,Import Loaction,Meal,-50
1,08/21/2019 16:49:48,Import Loaction,Meal,-10
2,08/23/2019 12:22:01,OHill ValuPort III,Debit,$-20.00
3,08/23/2019 12:22:08,OHill ValuPort III,Debit,$-20.00
4,08/23/2019 12:22:23,OHill ValuPort III,Debit,$-20.00
5,08/23/2019 12:25:51,O-Hill Aero 1 Loc 473,Meal,1
6,08/23/2019 18:41:53,O-Hill Aero 1 Loc 473,Meal,1
7,08/24/2019 08:32:16,O-Hill Aero 1 Loc 473,Meal,1
8,08/24/2019 14:07:13,ChickFilA (4414),Debit,$7.49
9,08/24/2019 18:35:32,O-Hill Aero 2 Loc 474,Meal,1


In [11]:
df['Amount'].value_counts()

1          308
$1.75       20
$-20.00      3
$7.75        2
$7.49        1
-50          1
$12.00       1
$10.08       1
-10          1
Name: Amount, dtype: int64

In [5]:
len(rows)

340

In [12]:
rows[0]

<tr class="heading">
<th>Post Date</th><th>Location</th><th>Account</th>
<th>Type</th> <th>Amount</th>
</tr>

In [13]:
rows[1]

<tr class="row1">
<td rowspan="2" width="140">08/21/2019 16:40:21</td>
<td rowspan="2" width="120">Import Loaction</td>
<td width="160">
<a href="javascript:void(0)" onclick="QuickHistory( 'M2' );">All Access 7</a>
</td>
<td width="80">
					Meal
									</td> <td align="right" width="60">-50</td>
</tr>

In [14]:
rows[2]

<tr class="row1">
<td width="160">
<a href="javascript:void(0)" onclick="QuickHistory( 'S31' );">Plus $</a>
</td>
<td width="80">
					Debit
									</td> <td align="right" width="60">$-150.00</td>
</tr>